# Check that we have everything here

In [ ]:
import os
if not os.path.isdir("quantificationlib"):
    print("You should have the quantification library in this directory")
    raise StopExecution

# Training the quantification model

In [85]:
import sys,os
import numpy as np

sys.path.insert(0, os.path.abspath("quantificationlib"))
from quantificationlib import classify_and_count

#Load the data
trainpreds = np.genfromtxt('results/trainpred.csv', delimiter=',')
traintrue = np.genfromtxt('results/traintrue.csv', delimiter=',')
trainprobs = np.genfromtxt('results/trainprobs.csv', delimiter=',')

quantifierCC = classify_and_count.CC(verbose=1)
quantifierCC.fit(None,traintrue,predictions_train=trainpreds)

Class CC: Computing predictions for training distribution...done


CC(estimator_test=None, verbose=1)

# Receiving the sample to quantify
Here I need to create a custom dataset to load the images from a folder without structure (no labels in this case)

In [87]:
import torchvision
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from natsort import natsorted

class ProductionDataset(Dataset):
    def __init__(self, main_dir, transform):
        self.main_dir = main_dir
        self.transform = transform
        all_imgs = os.listdir(main_dir)
        self.total_imgs = natsorted(all_imgs)

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = Image.open(img_loc).convert("RGB")
        tensor_image = self.transform(image)
        return tensor_image

prod_transform = T.Compose([
  T.Resize(size=256),
  T.CenterCrop(size=224),
  T.ToTensor(),
  #T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

#This directory should be the directory with the new images... using validation for simplicity here
prod_dset = ProductionDataset("production", transform=prod_transform)
prod_loader = DataLoader(prod_dset,batch_size=256,num_workers=4)
print("Loaded %d images " % len(prod_dset))

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50.]
Loaded 1967 images 


# Adding all neccesary functions

In this case the make prediction function does not use y (because they are not known). Also the load_network always needs a model.pt file (We have ran finetuning before)

In [64]:
import torch.nn as nn
import torch.nn.functional as nnf
from PIL import Image

def load_network():
  num_classes=51
  model = torchvision.models.resnet34(pretrained=True)
  print("Adjusting the CNN for %s classes" % num_classes)
  model.fc = nn.Linear(model.fc.in_features, num_classes)
  #Define loss function
  loss_fn = nn.CrossEntropyLoss()
  model.load_state_dict(torch.load("model.pt"))
  model = model.to(device) #Send model to gpu
  return model,loss_fn

def make_preds(model, loader, device):
  """
  Check the accuracy of the model.
  """
  with torch.no_grad():
    # Set the model to eval mode
    model.eval()
    y_pred = []
    y_probs = []
    for x in loader:
      x = x.to(device)
      # Run the model forward, and compare the argmax score with the ground-truth
      # category.
      output = model(x)
      predicted = output.argmax(1)
      prob = nnf.softmax(output, dim=1)
      y_probs.extend(prob.cpu().detach().numpy())
      y_pred.extend(predicted.cpu().numpy())
  return y_pred,y_probs

# Classify the sample and then quantify it

In [88]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using %s"%device)

model,_ =  load_network()
y_pred,y_probs = make_preds(model, prod_loader, device)
y_pred=np.vstack( y_pred )
print(quantifierCC.predict(None,predictions_test=y_pred))

Using cuda:0
Adjusting the CNN for 51 classes
Class CC: Computing predictions for testing distribution...done
Class CC: Computing prevalences for testing distribution...done
[2.03355363e-03 0.00000000e+00 5.08388409e-04 6.60904931e-03
 1.52516523e-03 1.01677682e-03 9.15099136e-03 5.08388409e-04
 5.08388409e-04 5.08388409e-04 0.00000000e+00 0.00000000e+00
 3.55871886e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 9.15099136e-03 1.11845450e-02 5.08388409e-04 5.08388409e-04
 1.01677682e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 8.43924759e-02 0.00000000e+00 5.08388409e-04 0.00000000e+00
 0.00000000e+00 0.00000000e+00 5.08388409e-04 0.00000000e+00
 5.08388409e-04 0.00000000e+00 5.08388409e-04 4.06710727e-03
 0.00000000e+00 0.00000000e+00 1.37264870e-02 2.54194204e-03
 0.00000000e+00 3.05033045e-03 0.00000000e+00 2.37417387e-01
 2.03355363e-03 0.00000000e+00 5.95831215e-01 4.06710727e-03
 1.52516523e-03 5.08388409e-04 5.08388409e-04]
